In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2
from sklearn.decomposition import PCA
from google.colab.patches import cv2_imshow
from numpy.linalg import eigh

In [2]:
count=0
dir="/content/drive/My Drive/assg3_q1_dataset/"
img = cv2.imread("/content/drive/My Drive/assg3_q1_dataset/002_048.jpg")
img = cv2.resize(img, (0,0), fx=0.125, fy=0.125) 
grayall = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
grayall=grayall.flatten()
labels=['2']
print(len( os.listdir(dir)))
for filename in os.listdir(dir):
    count+=1
    # if count==500:
    #     break
    # print(count,end=",")
    
    img = cv2.imread(dir+filename)
    if img is None:
        continue
    labels.append(filename[2])
    # print(count,end=" ")
    img = cv2.resize(img, (0,0), fx=0.125, fy=0.125) 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray=gray.flatten()
    grayall=np.append(grayall,gray)
grayall=grayall.reshape(521,-1)
print(len(labels))
grayall.shape

521
521


(521, 1024)

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(grayall,labels, test_size=0.25, random_state=0)
print(x_test.shape)
print(len(y_test))

(131, 1024)
131


In [0]:

def calc_pca(data,num_eigen):
	exit = []
	previous = []
	mean = np.mean(data, axis = 0)
    # print(mean.shape)
    # print(mean)
	mean_sub = data - mean
    # print(mean_sub.shape)
    # print(mean_sub)
	cov = np.cov(mean_sub, rowvar = 0)
    # print(cov.shape)
    # print(cov)
	eigen_values, eigen_vectors = np.linalg.eig(np.mat(cov))
	selection=0
	for i in range(num_eigen):
		selection+=eigen_values[i]
	
	print(selection/sum(eigen_values))
	eigen_values_sorted = np.argsort(-eigen_values)
	# print(eigen_vectors.shape)

	eigen_values_filtered = eigen_values_sorted[:num_eigen]
	eigen_vectors_filtered = eigen_vectors[:,eigen_values_filtered]
	# print(eigen_vectors_filtered.shape)
	data_matrix = mean_sub * eigen_vectors_filtered		
	reconstructed = (data_matrix * eigen_vectors_filtered.T) + mean
	return (data_matrix, reconstructed.real, eigen_vectors_filtered.T)
	

In [0]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=10)
# pca.fit(x_train)
# print(pca.components_.shape)

In [6]:
print(x_train.shape)
fitpca=calc_pca(x_train,100)
x_train=fitpca[0].real
print(x_train.real)

print(x_test.shape)
mean = np.mean(x_test, axis = 0)
# print(mean.shape)
# print(mean)
mean_sub = x_test - mean
x_test = mean_sub * fitpca[2].T
x_test=x_test.real
print(x_test.shape)
# reconstructed = (data_matrix * ans[2]) + mean
# img=reconstructed.real[0]
# cv2_imshow(img.reshape(32,32))

(390, 1024)
(0.9325935540414743+0j)
[[-5.61510336e+02  1.26203972e+03 -1.20928547e+02 ...  4.74808047e+01
   3.70671705e+01  1.56535147e+00]
 [-4.52204363e+01  3.20911630e+02 -4.72429278e+02 ... -3.19651875e+01
   1.34771175e+01  3.50523534e+01]
 [-1.01829089e+03 -8.18026630e+02  1.01400311e+02 ... -1.14092141e+02
  -4.11978666e+01 -6.04163187e+01]
 ...
 [-6.09593120e+02  1.05555737e+03  8.88054312e+00 ...  1.67696761e+01
   4.19025372e+01 -2.61321526e+01]
 [ 9.34060079e+02 -6.21246611e+02  2.44270480e+02 ...  2.96734464e+01
   5.07101700e+00 -2.05886507e+01]
 [-7.63613581e+02  2.04716837e+02 -3.34184375e+01 ...  6.97441277e-01
  -5.28843721e+01  4.28243906e+01]]
(131, 1024)
(131, 100)


In [0]:
# #ans[a][b][c] where a is sample(no of eigen vec), c is one image or row

# cv2_imshow(ans[1][0].reshape(32,32)) #keeping 5 features
# cv2_imshow(ans[1][0].reshape(32,32)) #keeping 25 features
# # cv2_imshow(ans[2][1][0].reshape(32,32)) #keeping 50 features

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(x_train)
# Apply transform to both the training set and the test set.
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(max_iter=10000,multi_class='ovr')
logisticRegr.fit(x_train, y_train)
y_pred=logisticRegr.predict(x_test)
print(y_pred)


['6' '2' '2' '5' '3' '3' '5' '6' '4' '6' '0' '5' '2' '1' '5' '4' '3' '7'
 '5' '6' '1' '6' '0' '0' '1' '0' '7' '4' '3' '1' '5' '0' '2' '5' '3' '6'
 '7' '7' '7' '5' '2' '1' '1' '4' '2' '4' '2' '4' '0' '3' '3' '4' '6' '7'
 '3' '1' '4' '7' '4' '7' '0' '5' '6' '1' '3' '7' '3' '1' '4' '7' '3' '0'
 '6' '0' '5' '2' '2' '6' '5' '7' '0' '2' '7' '2' '0' '4' '7' '2' '5' '6'
 '3' '1' '2' '4' '1' '5' '1' '7' '2' '2' '7' '6' '6' '0' '2' '2' '6' '5'
 '0' '2' '0' '1' '3' '4' '4' '6' '0' '1' '7' '3' '7' '1' '0' '6' '0' '5'
 '5' '1' '7' '5' '2']


In [10]:

import sklearn 
print(sklearn.metrics.accuracy_score(y_test, y_pred, normalize=True, sample_weight=None))
print(y_test)
score = logisticRegr.score(x_test, y_test)
print(score)

0.5801526717557252
['6', '2', '2', '5', '3', '7', '5', '4', '4', '0', '7', '5', '2', '7', '4', '3', '3', '7', '5', '4', '0', '6', '1', '0', '1', '5', '7', '4', '3', '1', '5', '1', '6', '5', '5', '6', '0', '7', '7', '5', '2', '4', '3', '4', '1', '5', '4', '4', '7', '3', '3', '4', '6', '3', '3', '1', '4', '7', '4', '0', '0', '4', '6', '1', '3', '6', '7', '6', '4', '7', '3', '0', '6', '3', '1', '2', '2', '1', '5', '7', '0', '2', '4', '2', '0', '6', '7', '2', '5', '7', '5', '1', '2', '4', '1', '5', '3', '7', '1', '2', '7', '6', '2', '4', '7', '1', '2', '5', '7', '5', '0', '1', '3', '4', '0', '0', '1', '5', '7', '3', '4', '6', '7', '6', '4', '1', '5', '3', '7', '3', '1']
0.5801526717557252


In [0]:
import numpy as np
import matplotlib.pyplot as plt

class MultiClassLogisticRegression:
    
    def __init__(self, n_iter = 10000, thres=1e-3):
        self.n_iter = n_iter
        self.thres = thres
    
    def fit(self, X, y, batch_size=64, lr=0.001, rand_seed=4, verbose=False): 
        np.random.seed(rand_seed) 
        self.classes = np.unique(y)
        self.class_labels = {c:i for i,c in enumerate(self.classes)}
        X = self.add_bias(X)
        y = self.one_hot(y)
        self.loss = []
        self.weights = np.zeros(shape=(len(self.classes),X.shape[1]))
        self.fit_data(X, y, batch_size, lr, verbose)
        return self
 
    def fit_data(self, X, y, batch_size, lr, verbose):
        i = 0
        while (not self.n_iter or i < self.n_iter):
            self.loss.append(self.cross_entropy(y, self.predict_(X)))
            idx = np.random.choice(X.shape[0], batch_size)
            X_batch, y_batch = X[idx], y[idx]
            error = y_batch - self.predict_(X_batch)
            update = (lr * np.dot(error.T, X_batch))
            self.weights += update
            if np.abs(update).max() < self.thres: break
            if i % 1000 == 0 and verbose: 
                print(' Training Accuray at {} iterations is {}'.format(i, self.evaluate_(X, y)))
            i +=1
    
    def predict(self, X):
        return self.predict_(self.add_bias(X))
    
    def predict_(self, X):
        pre_vals = np.dot(X, self.weights.T).reshape(-1,len(self.classes))
        return self.softmax(pre_vals)
    
    def softmax(self, z):
        return np.exp(z) / np.sum(np.exp(z), axis=1).reshape(-1,1)

    def predict_classes(self, X):
        self.probs_ = self.predict(X)
        return np.vectorize(lambda c: self.classes[c])(np.argmax(self.probs_, axis=1))
  
    def add_bias(self,X):
        return np.insert(X, 0, 1, axis=1)
  
    def get_randon_weights(self, row, col):
        return np.zeros(shape=(row,col))

    def one_hot(self, y):
        return np.eye(len(self.classes))[np.vectorize(lambda c: self.class_labels[c])(y).reshape(-1)]
    
    def score(self, X, y):
        return np.mean(self.predict_classes(X) == y)
    
    def evaluate_(self, X, y):
        return np.mean(np.argmax(self.predict_(X), axis=1) == np.argmax(y, axis=1))
    
    def cross_entropy(self, y, probs):
        return -1 * np.mean(y * np.log(probs))

In [12]:

lr = MultiClassLogisticRegression()

lr.fit(x_train,y_train, lr=0.0005)

pre = lr.predict_classes(x_test)
print(pre)
print(y_test)
import sklearn
print(sklearn.metrics.accuracy_score(pre, y_test))

['6' '2' '2' '4' '5' '3' '5' '6' '4' '1' '7' '5' '2' '7' '5' '4' '3' '7'
 '5' '1' '1' '6' '0' '0' '1' '0' '7' '4' '3' '1' '5' '4' '2' '5' '1' '0'
 '7' '6' '7' '5' '2' '4' '1' '4' '2' '4' '2' '4' '0' '3' '3' '4' '6' '7'
 '3' '0' '4' '7' '4' '7' '0' '5' '6' '1' '1' '7' '7' '1' '4' '7' '3' '0'
 '6' '1' '0' '2' '2' '1' '5' '7' '0' '2' '4' '2' '0' '4' '7' '2' '5' '6'
 '0' '1' '2' '4' '1' '5' '1' '7' '2' '2' '7' '6' '6' '0' '2' '2' '6' '5'
 '0' '2' '0' '1' '3' '4' '4' '6' '0' '1' '7' '3' '1' '5' '0' '6' '0' '5'
 '5' '1' '7' '5' '2']
['6', '2', '2', '5', '3', '7', '5', '4', '4', '0', '7', '5', '2', '7', '4', '3', '3', '7', '5', '4', '0', '6', '1', '0', '1', '5', '7', '4', '3', '1', '5', '1', '6', '5', '5', '6', '0', '7', '7', '5', '2', '4', '3', '4', '1', '5', '4', '4', '7', '3', '3', '4', '6', '3', '3', '1', '4', '7', '4', '0', '0', '4', '6', '1', '3', '6', '7', '6', '4', '7', '3', '0', '6', '3', '1', '2', '2', '1', '5', '7', '0', '2', '4', '2', '0', '6', '7', '2', '5', '7', '5', '1', '2', '